# HW2 Problem 3 Job Scheduling

In [1]:
import pandas as pd
import numpy as np

df = pd.DataFrame({"Job":[i for i in range(1,16)],
                  "ProcessingTime":[23,14,7,62,21,17,19,55,16,13,27,31,35,18,9]})

p = df.ProcessingTime.tolist()
p

[23, 14, 7, 62, 21, 17, 19, 55, 16, 13, 27, 31, 35, 18, 9]

In [2]:
import os
import pyomo.environ as pyo
from pyomo.environ import *

from pyomo.opt import SolverFactory

opt = pyo.SolverFactory('glpk')

model = ConcreteModel()

Jobs = p
ProcessingTime = df.values[1,:]
MaxSlots = 100

model.Cmax = Var(within=NonNegativeIntegers, initialize=0)
model.X = Var(range(4), range(len(Jobs)), range(MaxSlots), within=Binary, initialize=0)

model.obj = Objective(expr = model.Cmax, 
                      sense=minimize)
model.constraints = ConstraintList()

# Objective constraint smallest finish time
for i in model.X_index_0.data():
    for j in model.X_index_1.data():
        model.constraints.add(sum([model.X[i,j,t]*(t + p[j]) for t in model.X_index_2.data()]) <= model.Cmax)

# If Xijt selected, the next pj must be 0
#for i in model.X_index_0.data():
#    for j in model.X_index_1.data():
#        for t in model.X_index_2.data():
#            if t+p[j] <= MaxSlots:
#                model.constraints.add(sum([model.X[i,j,s] for s in range(t+1, t+p[j])]) <= (1-model.X[i,j,t]) )
#            else:
#                model.constraints.add(sum([model.X[i,j,s] for s in range(t+1, MaxSlots)]) <= (1-model.X[i,j,t]) )           

# Once a machine starts a job it can't start another job until it is finished
# For every machine (include yourself), if a job is selected then the total number of you/other machines and jobs taken during the next t+1 .. t+pj slots must be 0
for i in model.X_index_0.data():
    for itick in model.X_index_0.data():
        for jtick in model.X_index_1.data():
            for t in model.X_index_2.data():
                if t+p[j] <= MaxSlots:
                    model.constraints.add(sum([model.X[itick,j,s] for s in range(t+1, t+p[j]) for j in model.X_index_1.data()]) <= (1 - model.X[i,jtick,t]) )
                else:
                    model.constraints.add(sum([model.X[itick,j,s] for s in range(t+1, MaxSlots) for j in model.X_index_1.data()]) <= (1 - model.X[i,jtick,t]) )          

# Total jobs being worked by machine at a time
for i in model.X_index_0.data():
    for t in model.X_index_2.data():
        model.constraints.add(sum([model.X[i,j,t] for j in model.X_index_1.data()]) <= 1 )

# Total machines working on a job at a time
for j in model.X_index_1.data():
    for t in model.X_index_2.data():
        model.constraints.add(sum([model.X[i,j,t] for i in model.X_index_0.data()]) <= 1 )
        

# If a job starts it must be finished before MaxSlots
#for i in model.X_index_0.data():
#    for j in model.X_index_1.data():
#        for t in model.X_index_2.data():
#            model.constraints.add(model.X[i,j,t](t + p[j]) <= model.X[i,j,t]*MaxSlots )

# Every job must get done
for j in model.X_index_1.data():
    model.constraints.add(sum([model.X[i,j,t] for i in model.X_index_0.data() for t in model.X_index_2.data()]) >= 1)
#model.constraints.add(sum([model.X[i,j,t] for i in model.X_index_0.data() for t in model.X_index_2.data() for j in model.X_index_1.data()]) >= len(Jobs))



# Solution

In [ ]:
# Create a model instance and optimize
instance = model.create_instance()
results = opt.solve(instance, tee=True)
instance.display()


    model; returning a clone of the current model instance.
GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write C:\Users\bmccs\AppData\Local\Temp\tmp7cum0xwe.glpk.raw --wglp C:\Users\bmccs\AppData\Local\Temp\tmp67wrv3yu.glpk.glp
 --cpxlp C:\Users\bmccs\AppData\Local\Temp\tmpxck69jnp.pyomo.lp
Reading problem data from 'C:\Users\bmccs\AppData\Local\Temp\tmpxck69jnp.pyomo.lp'...
C:\Users\bmccs\AppData\Local\Temp\tmpxck69jnp.pyomo.lp:2882402: warning: lower bound of variable 'x2' redefined
C:\Users\bmccs\AppData\Local\Temp\tmpxck69jnp.pyomo.lp:2882402: warning: upper bound of variable 'x2' redefined
25976 rows, 6002 columns, 2798461 non-zeros
6001 integer variables, 6000 of which are binary
2888402 lines were read
Writing problem data to 'C:\Users\bmccs\AppData\Local\Temp\tmp67wrv3yu.glpk.glp'...
2856421 lines were written
GLPK Integer Optimizer, v4.65
25976 rows, 6002 columns, 2798461 non-zeros
6001 integer variables, 6000 of which are binary
Preprocessi

+  4657: mip =     not found yet >=   1.600000000e+01        (88; 0)
+  4678: mip =     not found yet >=   1.600000000e+01        (89; 0)
Time used: 566.9 secs.  Memory used: 323.5 Mb.
+  4693: mip =     not found yet >=   1.600000000e+01        (90; 0)
+  4808: mip =     not found yet >=   1.600000000e+01        (91; 0)
+  4906: mip =     not found yet >=   1.600000000e+01        (92; 0)
+  5007: mip =     not found yet >=   1.600000000e+01        (93; 0)
+  5119: mip =     not found yet >=   1.600000000e+01        (94; 0)
+  5314: mip =     not found yet >=   1.600000000e+01        (95; 0)
+  5452: mip =     not found yet >=   1.600000000e+01        (96; 0)
+  5552: mip =     not found yet >=   1.600000000e+01        (97; 0)
+  5648: mip =     not found yet >=   1.600000000e+01        (98; 0)
Time used: 633.6 secs.  Memory used: 329.4 Mb.
+  5697: mip =     not found yet >=   1.600000000e+01        (99; 0)
+  5782: mip =     not found yet >=   1.600000000e+01        (100; 0)
+  5868:

+ 11752: mip =     not found yet >=   1.600000000e+01        (196; 0)
+ 11815: mip =     not found yet >=   1.600000000e+01        (197; 0)
+ 11896: mip =     not found yet >=   1.600000000e+01        (198; 0)
+ 11916: mip =     not found yet >=   1.600000000e+01        (199; 0)
+ 11998: mip =     not found yet >=   1.600000000e+01        (200; 0)
Time used: 1537.5 secs.  Memory used: 329.6 Mb.
+ 12089: mip =     not found yet >=   1.600000000e+01        (201; 0)
+ 12090: mip =     not found yet >=   1.600000000e+01        (202; 0)
+ 12163: mip =     not found yet >=   1.600000000e+01        (203; 0)
+ 12273: mip =     not found yet >=   1.600000000e+01        (204; 0)
+ 12338: mip =     not found yet >=   1.600000000e+01        (205; 0)
+ 12442: mip =     not found yet >=   1.600000000e+01        (206; 0)
+ 12529: mip =     not found yet >=   1.600000000e+01        (207; 0)
Time used: 1605.4 secs.  Memory used: 329.6 Mb.
+ 12641: mip =     not found yet >=   1.600000000e+01        (20

+ 18800: mip =     not found yet >=   1.600000000e+01        (303; 0)
+ 18831: mip =     not found yet >=   1.600000000e+01        (304; 0)
+ 18915: mip =     not found yet >=   1.600000000e+01        (305; 0)
Time used: 2524.4 secs.  Memory used: 329.8 Mb.
+ 18980: mip =     not found yet >=   1.600000000e+01        (306; 0)
+ 19055: mip =     not found yet >=   1.600000000e+01        (307; 0)
+ 19142: mip =     not found yet >=   1.600000000e+01        (308; 0)
+ 19180: mip =     not found yet >=   1.600000000e+01        (309; 0)
+ 19256: mip =     not found yet >=   1.600000000e+01        (310; 0)
+ 19332: mip =     not found yet >=   1.600000000e+01        (311; 0)
+ 19353: mip =     not found yet >=   1.600000000e+01        (312; 0)
Time used: 2592.8 secs.  Memory used: 329.8 Mb.
+ 19393: mip =     not found yet >=   1.600000000e+01        (313; 0)
+ 19498: mip =     not found yet >=   1.600000000e+01        (314; 0)
+ 19575: mip =     not found yet >=   1.600000000e+01        (31

+ 27554: mip =     not found yet >=   1.600000000e+01        (408; 0)
+ 27723: mip =     not found yet >=   1.600000000e+01        (409; 0)
+ 27877: mip =     not found yet >=   1.600000000e+01        (410; 0)
+ 27878: mip =     not found yet >=   1.600000000e+01        (411; 0)
Time used: 3703.5 secs.  Memory used: 330.1 Mb.
+ 27879: mip =     not found yet >=   1.600000000e+01        (412; 0)
+ 27880: mip =     not found yet >=   1.600000000e+01        (413; 0)
+ 27930: mip =     not found yet >=   1.600000000e+01        (414; 0)
+ 28050: mip =     not found yet >=   1.600000000e+01        (415; 0)
+ 28283: mip =     not found yet >=   1.600000000e+01        (416; 0)
Time used: 3764.0 secs.  Memory used: 330.1 Mb.
+ 28359: mip =     not found yet >=   1.600000000e+01        (417; 0)
+ 28487: mip =     not found yet >=   1.600000000e+01        (418; 0)
+ 28567: mip =     not found yet >=   1.600000000e+01        (419; 0)
+ 28668: mip =     not found yet >=   1.600000000e+01        (42

...

+768199: mip =   1.160000000e+02 >=   1.700000000e+01  85.3% (6147; 548)
+768248: mip =   1.160000000e+02 >=   1.700000000e+01  85.3% (6148; 548)
+768342: mip =   1.160000000e+02 >=   1.700000000e+01  85.3% (6149; 548)
+768372: mip =   1.160000000e+02 >=   1.700000000e+01  85.3% (6150; 548)
Time used: 59065.8 secs.  Memory used: 363.3 Mb.
+768486: mip =   1.160000000e+02 >=   1.700000000e+01  85.3% (6151; 548)
+768551: mip =   1.160000000e+02 >=   1.700000000e+01  85.3% (6152; 548)
+768748: mip =   1.160000000e+02 >=   1.700000000e+01  85.3% (6153; 548)
+768866: mip =   1.160000000e+02 >=   1.700000000e+01  85.3% (6154; 548)
+768975: mip =   1.160000000e+02 >=   1.700000000e+01  85.3% (6155; 548)
+769234: mip =   1.160000000e+02 >=   1.700000000e+01  85.3% (6156; 548)
Time used: 59128.8 secs.  Memory used: 363.3 Mb.
+769350: mip =   1.160000000e+02 >=   1.700000000e+01  85.3% (6157; 548)
+769351: mip =   1.160000000e+02 >=   1.700000000e+01  85.3% (6158; 548)
+769352: mip =   1.1600000

+778385: mip =   1.160000000e+02 >=   1.700000000e+01  85.3% (6248; 548)
+778476: mip =   1.160000000e+02 >=   1.700000000e+01  85.3% (6249; 548)
+778650: mip =   1.160000000e+02 >=   1.700000000e+01  85.3% (6250; 548)
+778804: mip =   1.160000000e+02 >=   1.700000000e+01  85.3% (6251; 548)
+778957: mip =   1.160000000e+02 >=   1.700000000e+01  85.3% (6252; 548)
Time used: 60185.7 secs.  Memory used: 363.6 Mb.
+779133: mip =   1.160000000e+02 >=   1.700000000e+01  85.3% (6253; 548)
+779244: mip =   1.160000000e+02 >=   1.700000000e+01  85.3% (6254; 548)
+779371: mip =   1.160000000e+02 >=   1.700000000e+01  85.3% (6255; 548)
+779499: mip =   1.160000000e+02 >=   1.700000000e+01  85.3% (6256; 548)
+779678: mip =   1.160000000e+02 >=   1.700000000e+01  85.3% (6257; 548)
+779809: mip =   1.160000000e+02 >=   1.700000000e+01  85.3% (6258; 548)
Time used: 60254.7 secs.  Memory used: 363.7 Mb.
+780033: mip =   1.160000000e+02 >=   1.700000000e+01  85.3% (6259; 548)
+780207: mip =   1.1600000

+794690: mip =   1.160000000e+02 >=   1.700000000e+01  85.3% (6350; 548)
+794841: mip =   1.160000000e+02 >=   1.700000000e+01  85.3% (6351; 548)
+795035: mip =   1.160000000e+02 >=   1.700000000e+01  85.3% (6352; 548)
+795290: mip =   1.160000000e+02 >=   1.700000000e+01  85.3% (6353; 548)
+795498: mip =   1.160000000e+02 >=   1.700000000e+01  85.3% (6354; 548)
+795595: mip =   1.160000000e+02 >=   1.700000000e+01  85.3% (6355; 548)
Time used: 61212.9 secs.  Memory used: 364.1 Mb.
+795812: mip =   1.160000000e+02 >=   1.700000000e+01  85.3% (6356; 548)
+796019: mip =   1.160000000e+02 >=   1.700000000e+01  85.3% (6357; 548)
+796181: mip =   1.160000000e+02 >=   1.700000000e+01  85.3% (6358; 548)
+796312: mip =   1.160000000e+02 >=   1.700000000e+01  85.3% (6359; 548)
+796428: mip =   1.160000000e+02 >=   1.700000000e+01  85.3% (6360; 548)
+796600: mip =   1.160000000e+02 >=   1.700000000e+01  85.3% (6361; 548)
+796669: mip =   1.160000000e+02 >=   1.700000000e+01  85.3% (6362; 548)
Ti

+808261: mip =   1.160000000e+02 >=   1.700000000e+01  85.3% (6453; 548)
+808531: mip =   1.160000000e+02 >=   1.700000000e+01  85.3% (6454; 548)
+808843: mip =   1.160000000e+02 >=   1.700000000e+01  85.3% (6455; 548)
Time used: 62199.9 secs.  Memory used: 364.4 Mb.
+809134: mip =   1.160000000e+02 >=   1.700000000e+01  85.3% (6456; 548)
+809393: mip =   1.160000000e+02 >=   1.700000000e+01  85.3% (6457; 548)
+809684: mip =   1.160000000e+02 >=   1.700000000e+01  85.3% (6458; 548)
+809801: mip =   1.160000000e+02 >=   1.700000000e+01  85.3% (6459; 548)
+810076: mip =   1.160000000e+02 >=   1.700000000e+01  85.3% (6460; 548)
+810212: mip =   1.160000000e+02 >=   1.700000000e+01  85.3% (6461; 548)
+810381: mip =   1.160000000e+02 >=   1.700000000e+01  85.3% (6462; 548)
Time used: 62268.0 secs.  Memory used: 364.5 Mb.
+810503: mip =   1.160000000e+02 >=   1.700000000e+01  85.3% (6463; 548)
+810728: mip =   1.160000000e+02 >=   1.700000000e+01  85.3% (6464; 548)
+810811: mip =   1.1600000

+822015: mip =   1.160000000e+02 >=   1.700000000e+01  85.3% (6556; 548)
+822027: mip =   1.160000000e+02 >=   1.700000000e+01  85.3% (6557; 548)
+822028: mip =   1.160000000e+02 >=   1.700000000e+01  85.3% (6558; 548)
+822029: mip =   1.160000000e+02 >=   1.700000000e+01  85.3% (6559; 548)
+822030: mip =   1.160000000e+02 >=   1.700000000e+01  85.3% (6560; 548)
+822031: mip =   1.160000000e+02 >=   1.700000000e+01  85.3% (6561; 548)
Time used: 63251.3 secs.  Memory used: 364.8 Mb.
+822032: mip =   1.160000000e+02 >=   1.700000000e+01  85.3% (6562; 548)
+822033: mip =   1.160000000e+02 >=   1.700000000e+01  85.3% (6563; 548)
+822183: mip =   1.160000000e+02 >=   1.700000000e+01  85.3% (6564; 548)
+822293: mip =   1.160000000e+02 >=   1.700000000e+01  85.3% (6565; 548)
+822406: mip =   1.160000000e+02 >=   1.700000000e+01  85.3% (6566; 548)
+822451: mip =   1.160000000e+02 >=   1.700000000e+01  85.3% (6567; 548)
Time used: 63315.3 secs.  Memory used: 364.8 Mb.
+822592: mip =   1.1600000

In [52]:
instance.Cmax.get_values()

{None: 62.0}

In [53]:
for k,v in instance.X.get_values().items():
    if v == 1:
        print(k,v)

(0, 0, 2) 1.0
(0, 1, 29) 1.0
(0, 2, 11) 1.0
(0, 2, 20) 1.0
(0, 5, 42) 1.0
(1, 0, 19) 1.0
(1, 3, 0) 1.0
(1, 4, 28) 1.0
(1, 8, 37) 1.0
(1, 12, 10) 1.0
(2, 9, 1) 1.0
(2, 10, 21) 1.0
(2, 11, 30) 1.0
(2, 13, 12) 1.0
(3, 6, 32) 1.0
(3, 7, 2) 1.0
(3, 14, 19) 1.0


In [37]:
D = np.zeros((50, 4, 15))
for k,v in instance.X.get_values().items():
    if v == 1:
        D[k[2],k[0],k[1]] = 1
for t in range(D.shape[0]):
    if D[t,:,:].sum() > 0:
        print(t, D[t,:,:])

0 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
2 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
4 [[0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
7 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]]
8 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
13 [[0. 0. 0. 0. 1. 0. 0.

[23, 14, 7, 62, 21, 17, 19, 55, 16, 13, 27, 31, 35, 18, 9]